## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,49.87,91,100,10.60,light rain
1,1,Torbay,CA,47.6666,-52.7314,52.75,99,90,21.85,light rain
2,2,Rikitea,PF,-23.1203,-134.9692,76.87,72,8,10.40,clear sky
3,3,Tasiilaq,GL,65.6145,-37.6368,26.76,56,100,8.75,overcast clouds
4,4,Neiafu,TO,-18.6500,-173.9833,84.88,70,75,6.91,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 95


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Neiafu,TO,-18.6500,-173.9833,84.88,70,75,6.91,broken clouds
6,6,Avarua,CK,-21.2078,-159.7750,82.45,78,45,9.22,light rain
7,7,Butaritari,KI,3.0707,172.7902,82.74,70,100,16.62,overcast clouds
8,8,Vaini,TO,-21.2000,-175.2000,86.16,84,75,8.05,light rain
18,18,Kavaratti,IN,10.5669,72.6420,81.99,72,100,1.83,overcast clouds
19,19,Kapaa,US,22.0752,-159.3190,84.18,72,20,1.99,light rain
31,31,Kahului,US,20.8947,-156.4700,83.46,72,40,20.71,scattered clouds
32,32,Faanui,PF,-16.4833,-151.7500,82.11,72,11,15.39,few clouds
41,41,Morant Bay,JM,17.8815,-76.4093,80.65,81,53,12.64,broken clouds
61,61,Georgetown,MY,5.4112,100.3354,81.90,86,20,1.01,few clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                92
City                   92
Country                92
Lat                    92
Lng                    92
Max Temp               92
Humidity               92
Cloudiness             92
Wind Speed             92
Current Description    92
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Neiafu,TO,84.88,broken clouds,-18.6500,-173.9833,
6,Avarua,CK,82.45,light rain,-21.2078,-159.7750,
7,Butaritari,KI,82.74,overcast clouds,3.0707,172.7902,
8,Vaini,TO,86.16,light rain,-21.2000,-175.2000,
18,Kavaratti,IN,81.99,overcast clouds,10.5669,72.6420,
19,Kapaa,US,84.18,light rain,22.0752,-159.3190,
31,Kahului,US,83.46,scattered clouds,20.8947,-156.4700,
32,Faanui,PF,82.11,few clouds,-16.4833,-151.7500,
41,Morant Bay,JM,80.65,broken clouds,17.8815,-76.4093,
61,Georgetown,MY,81.90,few clouds,5.4112,100.3354,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Neiafu,TO,84.88,broken clouds,-18.6500,-173.9833,Mystic Sands Beachfront Accommodation
6,Avarua,CK,82.45,light rain,-21.2078,-159.7750,Paradise Inn
7,Butaritari,KI,82.74,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
8,Vaini,TO,86.16,light rain,-21.2000,-175.2000,Keleti Beach Resort
18,Kavaratti,IN,81.99,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
max_temp = clean_hotel_df["Max Temp"]
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))